In [2]:
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 95%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [3]:
import requests
import pandas as pd
import numpy as np
import copy
import json

from ast import literal_eval

import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel
from transformers import OwlViTProcessor, OwlViTForObjectDetection
from transformers import pipeline
from transformers import GPT2TokenizerFast
from PIL import Image

import pickle

In [4]:
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict

import sklearn.datasets as datasets
import sklearn.manifold as manifold

In [5]:
import openai
import os
import sys
import dotenv
from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity
import tiktoken

api_path="C:/Users/2hy03/gpt_test.env"
load_dotenv(api_path) # .env 파일에서 환경 변수 로드
api_key = os.getenv("OPENAI_API_KEY") # API 키 가져오기
openai.api_key=os.getenv("OPENAI_API_KEY") 

In [70]:
cur_os = sys.platform

# ChapGPT이용

- 3개의 chatgpt이용
      - 1개는 민원을 요약하는 것
      - 1개는 비슷한 유형의 민원을 찾는 것-> 답변생성
           -embedding을 기반으로 유사도를 계산해 비슷한 민원의 정보를 가져옴
           -민원 답변과 부서도 함께 loading
           -> 민원 부서 전처리를 진행해야 부서추천 기능 추가 가능

# 1.민원 요약

### 필요한 prompt 형태 설정

- 요약의 경우, 
       민원내용: "새로운 민원 input"
       상황: "###"
       요약: "###"

In [79]:
def print_summary(msg):
    completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=msg
                    )
    return completion['choices'][0]['message']['content']

In [94]:
query = input("새로운 민원: ")

새로운 민원: 중구청 뒤 불법 주차 단속 요청. 아침 저녁으로 중구청 뒷길을 차량으로 이용하고 있다. 신포동 일대의 주차 단속은 꽤 철저한 편입니다. 제가 아는 지인도 주차 구역이 아닌 곳에 주차 후 주차 위반으로 여러번 범칙금을 내는 것을 봤다. 주차 구역 이외의 지역에 주차 하는 것은 여러 사람에게 불편을 초래하는 일로 당연히 철저하게 단속되어야 . 하지만 중구청 뒷길은 왜 이중 주차 허용하는지 궁금. 매일 그 길을 이용하면서 차량들이 엉키고 큰 트럭이 힘겹게 후진을 하는 일이 다반사입니다. 가장 어이없는 일은 차량들이 엉켜서 극심한 정체 빚고 있었을 때 주차 단속을 나오신 분이 어떠한 차량도 주차 단속하지 않고 차 후진하라는 말만 하고선 가군요. 정말 화가  났다. 그 골목은 주차단속을 하지 않는 구역인가요이런 일이 하루이틀 저만의 문제가 라서 주변 사람들에게 이야기해 보니 그 중구청 뒷골목의 차량들 대부분이 중구청 직원 차량이라는 이야기 들었다그래서 단속하지 않는다는 이야기도 들었다. 유독 중구청 뒤만 단속하지 않는 것으로 보아 이런 이야기가 나올만 .. 물론 이것은 확인되지 않은 사실이지만  이것이 사실이라면 요즘같이 1인 1차량 시대에 직원들을 위한 주차장이 구비되어야 한다고 봅니다. 이러한 불편을 그 지역을 지나다니는 사람들이 떠안을 이유는 없다고 봅니다.  하나 이렇게 민원을 넣어 봤자 일주일 단속한 뒤  원상대로 돌아간다는 이야기도 들었다. 저가 아이핀 인증까지 힘겹게 받아가지고 여기에 구구절절 민원을 남기는 이유는 잠깐만 불편을 해소하고자 올리는 것이 아닙니다. 근본적으로 문제가 해결되어서 중구청 뒷길에 이중주차 문제 불법 주차 문제가 해결되길 원하는 바입니다. 매번 그 길을 이용하면서 불편함이 아주 극심. 중구 지역의 불법 주차 문제 단속하고 편리 도모하는 중구청에서 먼저 모범을 보여 주었으면 . 이 문제가 근본적으로 해결되지 않는다면 다른 곳에 도움을 받아 해결해 볼 방법을 도모하겠다.


In [97]:
messages = [
    {"role": "system", "content": "You are a bot that summarizes the complaints in Jung-gu, Incheon. summarize the situation and content of the new complaint. 모든 대답은 한글로."},
    {"role": "user", "content" :"새로운 민원:{} \n상황: \n요구사항: \n상황과 요구사항을 작성해".format(query)}
]


summary=print_summary(messages)

In [98]:
print(query)
print("\n")
print(summary)

중구청 뒤 불법 주차 단속 요청. 아침 저녁으로 중구청 뒷길을 차량으로 이용하고 있다. 신포동 일대의 주차 단속은 꽤 철저한 편입니다. 제가 아는 지인도 주차 구역이 아닌 곳에 주차 후 주차 위반으로 여러번 범칙금을 내는 것을 봤다. 주차 구역 이외의 지역에 주차 하는 것은 여러 사람에게 불편을 초래하는 일로 당연히 철저하게 단속되어야 . 하지만 중구청 뒷길은 왜 이중 주차 허용하는지 궁금. 매일 그 길을 이용하면서 차량들이 엉키고 큰 트럭이 힘겹게 후진을 하는 일이 다반사입니다. 가장 어이없는 일은 차량들이 엉켜서 극심한 정체 빚고 있었을 때 주차 단속을 나오신 분이 어떠한 차량도 주차 단속하지 않고 차 후진하라는 말만 하고선 가군요. 정말 화가  났다. 그 골목은 주차단속을 하지 않는 구역인가요이런 일이 하루이틀 저만의 문제가 라서 주변 사람들에게 이야기해 보니 그 중구청 뒷골목의 차량들 대부분이 중구청 직원 차량이라는 이야기 들었다그래서 단속하지 않는다는 이야기도 들었다. 유독 중구청 뒤만 단속하지 않는 것으로 보아 이런 이야기가 나올만 .. 물론 이것은 확인되지 않은 사실이지만  이것이 사실이라면 요즘같이 1인 1차량 시대에 직원들을 위한 주차장이 구비되어야 한다고 봅니다. 이러한 불편을 그 지역을 지나다니는 사람들이 떠안을 이유는 없다고 봅니다.  하나 이렇게 민원을 넣어 봤자 일주일 단속한 뒤  원상대로 돌아간다는 이야기도 들었다. 저가 아이핀 인증까지 힘겹게 받아가지고 여기에 구구절절 민원을 남기는 이유는 잠깐만 불편을 해소하고자 올리는 것이 아닙니다. 근본적으로 문제가 해결되어서 중구청 뒷길에 이중주차 문제 불법 주차 문제가 해결되길 원하는 바입니다. 매번 그 길을 이용하면서 불편함이 아주 극심. 중구 지역의 불법 주차 문제 단속하고 편리 도모하는 중구청에서 먼저 모범을 보여 주었으면 . 이 문제가 근본적으로 해결되지 않는다면 다른 곳에 도움을 받아 해결해 볼 방법을 도모하겠다.


상황: 중구청 뒷길이 이중주차와 불법주차로 인해 차량들이 엉키고

# 2. 민원답변 생성

## 임베딩

## >BERT-base-uncased

In [ ]:
#BERT기반->bert-base-uncased
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

# BERT 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# 문장 입력받기
sentence1 = "The quick brown fox jumps over the lazy dog"
sentence2 = "A quick brown fox jumps over a lazy dog"

# 토큰화 및 인코딩
inputs1 = tokenizer.encode_plus(sentence1, add_special_tokens=True, return_tensors='pt')
inputs2 = tokenizer.encode_plus(sentence2, add_special_tokens=True, return_tensors='pt')

# 문장 임베딩 벡터 생성
with torch.no_grad():
    embeddings1 = model(inputs1['input_ids'], token_type_ids=None)[0].mean(dim=1)
    embeddings2 = model(inputs2['input_ids'], token_type_ids=None)[0].mean(dim=1)

# 코사인 유사도 계산
similarity = F.cosine_similarity(embeddings1, embeddings2)
print(similarity.item())


## > HuggingFace embedding

- DistilBERT(Distilled BERT) 모델

    -BERT 모델의 경량화 버전
    
    -WordPiece 토크나이저 사용
    
    -BERT 모델보다 빠른 추론 속도와 적은 메모리 사용량
    
    -대용량 데이터가 없는 상황에서도 잘 작동함
    

In [71]:
model_path = f"D:/github" if cur_os.startswith('win') else None

In [72]:
minwon_metadata= pd.read_csv('minwon.csv', sep=",", dtype=str)
print(minwon_metadata.shape)
minwon_metadata.head()

(521, 4)


,title,content,answer,depart
0,주차장이 너무 어두워요,민원 내용:주차장이 어두워요중구 전동 25328 번지옆 전동쉼터 옆 공영주차장이 ...,1. 구정 발전에 관심을 가져주시는 귀하께 감사의 말씀을 드립니다.2. 민원 제기하...,도시개발국 교통운수과
1,뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요?,민원 내용:뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요? 거기 ...,1. 안녕하십니까? 귀하께서 상담민원을 통해 신청하신 민원에 대한 검토 결과를 다음...,국제도시건설국 농수산과
2,9100번 광역버스 배차간격 준수 요청 건,민원 내용:9100번 광역버스 배차간격 준수 요청 건집안 사정으로 서구에서 남동구 ...,불편사항이 해결됨,도시개발국 교통운수과
3,46번버스 신흥교통 정상이아닌 버스업체,민원 내용:46번버스 신흥교통 정상이아닌 버스업체46번버스 신흥교통 손님에게 막말하...,1. 교통 업무에 많은 관심과 애정을 가져주셔서 깊이 감사드립니다. 2. 귀하께서 ...,도시개발국 교통운수과
4,살려주세요 제발 요즘 가게 망하게 만드는 일들만 일어나네요..제발요(영종도 호텔),민원 내용: 요즘 가게 망하게 만드는 일들만 일어나네요..요(영종도 호텔)요즘 전...,1. 구정 발전을 위해 관심을 가져주시는 귀하께 깊은 감사를 드립니다.2. 모두가 ...,국제도시건설국 도시공원과


In [8]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-distilroberta-base-v2')

In [ ]:
minwon_metadata['hf_embeddings'] = minwon_metadata['content'].apply(lambda x : model.encode(x))
minwon_metadata.shape

In [ ]:
minwon_metadata.head()

In [123]:
minwon_metadata.to_csv('minwon_metadata_hfem.csv',encoding='utf-8-sig',index=False)

-----

## > OpenAI Embedding

In [73]:
minwon_metadata= pd.read_csv('minwon.csv', sep=",", dtype=str)
df=minwon_metadata

,title,content,answer,depart
0,주차장이 너무 어두워요,민원 내용:주차장이 어두워요중구 전동 25328 번지옆 전동쉼터 옆 공영주차장이 ...,1. 구정 발전에 관심을 가져주시는 귀하께 감사의 말씀을 드립니다.2. 민원 제기하...,도시개발국 교통운수과
1,뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요?,민원 내용:뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요? 거기 ...,1. 안녕하십니까? 귀하께서 상담민원을 통해 신청하신 민원에 대한 검토 결과를 다음...,국제도시건설국 농수산과
2,9100번 광역버스 배차간격 준수 요청 건,민원 내용:9100번 광역버스 배차간격 준수 요청 건집안 사정으로 서구에서 남동구 ...,불편사항이 해결됨,도시개발국 교통운수과
3,46번버스 신흥교통 정상이아닌 버스업체,민원 내용:46번버스 신흥교통 정상이아닌 버스업체46번버스 신흥교통 손님에게 막말하...,1. 교통 업무에 많은 관심과 애정을 가져주셔서 깊이 감사드립니다. 2. 귀하께서 ...,도시개발국 교통운수과
4,살려주세요 제발 요즘 가게 망하게 만드는 일들만 일어나네요..제발요(영종도 호텔),민원 내용: 요즘 가게 망하게 만드는 일들만 일어나네요..요(영종도 호텔)요즘 전...,1. 구정 발전을 위해 관심을 가져주시는 귀하께 깊은 감사를 드립니다.2. 모두가 ...,국제도시건설국 도시공원과


In [74]:
#token수 확인
## gpt-3.5-turbo 모델에 맞는 token encoding model은 cl100k_base이다.
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

#contents 컬럼의 토큰 수를 계산하여 새로운 컬럼으로 만들어 붙임
df['tokens'] = df.content.apply(lambda x: num_tokens_from_string(x, 'cl100k_base'))
#del df['Unnamed: 0']
df = df.set_index(keys='title')
df.head()

,content,answer,depart,tokens
title,,,,
주차장이 너무 어두워요,민원 내용:주차장이 어두워요중구 전동 25328 번지옆 전동쉼터 옆 공영주차장이 ...,1. 구정 발전에 관심을 가져주시는 귀하께 감사의 말씀을 드립니다.2. 민원 제기하...,도시개발국 교통운수과,85
뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요?,민원 내용:뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요? 거기 ...,1. 안녕하십니까? 귀하께서 상담민원을 통해 신청하신 민원에 대한 검토 결과를 다음...,국제도시건설국 농수산과,1097
9100번 광역버스 배차간격 준수 요청 건,민원 내용:9100번 광역버스 배차간격 준수 요청 건집안 사정으로 서구에서 남동구 ...,불편사항이 해결됨,도시개발국 교통운수과,555
46번버스 신흥교통 정상이아닌 버스업체,민원 내용:46번버스 신흥교통 정상이아닌 버스업체46번버스 신흥교통 손님에게 막말하...,1. 교통 업무에 많은 관심과 애정을 가져주셔서 깊이 감사드립니다. 2. 귀하께서 ...,도시개발국 교통운수과,236
살려주세요 제발 요즘 가게 망하게 만드는 일들만 일어나네요..제발요(영종도 호텔),민원 내용: 요즘 가게 망하게 만드는 일들만 일어나네요..요(영종도 호텔)요즘 전...,1. 구정 발전을 위해 관심을 가져주시는 귀하께 깊은 감사를 드립니다.2. 모두가 ...,국제도시건설국 도시공원과,1490


In [75]:
df['depart'].value_counts()

도시개발국 교통운수과       71
도시개발국 건축과         68
도시개발국 건설과         62
국제도시건설국 기반시설과     29
국제도시건설국 도시공원과     23
도시개발국 환경보호과       18
국제도시건설국 교통과       18
기획감사실             18
관광진흥실             17
국제도시행정국 친환경조성과    17
보건소 보건행정과         16
주민생활지원국 청소과       15
홍보체육진흥실           14
도시관리국 교통행정과       13
용유출장소             13
주민생활지원국 자원순환과     13
보건소 건강증진과         13
국제도시국 교통지적과       13
국제도시건설국 농수산과      13
국제도시국 기반시설과       12
도시재생국 도시개발과       12
운서동               11
주민생활지원국 환경관리과     11
주민생활지원국 위생과       11
Name: depart, dtype: int64

In [76]:
df.tokens.mean()

439.6621880998081

In [77]:
#토큰수1200개 이상 삭제
##토큰수가 1200개가 넘을 경우 요약이 제대로 진행되지 않음
df = df[df['tokens'] <= 1200]
df.head()

,content,answer,depart,tokens
title,,,,
주차장이 너무 어두워요,민원 내용:주차장이 어두워요중구 전동 25328 번지옆 전동쉼터 옆 공영주차장이 ...,1. 구정 발전에 관심을 가져주시는 귀하께 감사의 말씀을 드립니다.2. 민원 제기하...,도시개발국 교통운수과,85
뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요?,민원 내용:뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요? 거기 ...,1. 안녕하십니까? 귀하께서 상담민원을 통해 신청하신 민원에 대한 검토 결과를 다음...,국제도시건설국 농수산과,1097
9100번 광역버스 배차간격 준수 요청 건,민원 내용:9100번 광역버스 배차간격 준수 요청 건집안 사정으로 서구에서 남동구 ...,불편사항이 해결됨,도시개발국 교통운수과,555
46번버스 신흥교통 정상이아닌 버스업체,민원 내용:46번버스 신흥교통 정상이아닌 버스업체46번버스 신흥교통 손님에게 막말하...,1. 교통 업무에 많은 관심과 애정을 가져주셔서 깊이 감사드립니다. 2. 귀하께서 ...,도시개발국 교통운수과,236
신흥교통 46번버스회사 고발합니다,민원 내용:신흥교통 46번버스회사 고발46번버스이용중 불편사항 해당교통회사 전화문의...,1. 교통 업무에 많은 관심과 애정을 가져주셔서 깊이 감사드립니다. 2. 귀하께서 ...,도시개발국 교통운수과,178


---

### 임베딩 & 코사인유사도 함수 정의

In [63]:
#자연어->embedding벡터
##embedding 모델 : text-embedding-ada-00

COMPLETIONS_MODEL = "gpt-3.5-turbo"
EMBEDDING_MODEL = "text-embedding-ada-002"

def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
        # index는 그대로 유지하고 content에 있는 내용만 embedding vector로 바꾼 새로운 데이터 프레임 생성
    }

In [ ]:
#코사인 유사도

def vector_similarity(x: list[float], y: list[float]) -> float:
    
    return np.dot(np.array(x), np.array(y))

#새로운 입력값을 임베딩
def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:

    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

---

### 부서별 임베딩 실행 및 저장
    중간에 에러가 날 수 있으므로 나눠서 실행함

In [248]:
Environmental = df[df['depart'] == '운서동']
Environmental.head()

,content,answer,depart,tokens
title,,,,
영종도 운서역 종량제 10L 어디에 파나요 ?,민원 내용:영종도 운서역 종량제 10L 어디에 파나요 ?수고 많으십니다 종량제 10...,1. 우리 洞 발전에 관심을 가져주신 귀하께 감사의 말씀을 드립니다.2. 종량제봉투...,운서동,77
앞 공터 쓰레기더미를 치워 주세요!,민원 내용:앞 공터 쓰레기미 치워 주세요! 저는 저번주 일요일 부산에서 직장때문에 ...,1. 우리 洞 발전에 관심을 가져주신 귀하께 감사의 말씀을 드립니다. 2. 귀하께서...,운서동,232
운서동 두림타워 신축공사 민원,민원 내용:운서동 두림타워 신축공사 민원.두림타워 신축공사 앞 에어스테이호텔 운영...,1. 우리 洞 발전에 관심을 가져주신 귀하께 감사의 말씀을 드립니다.2. 귀하께서 ...,운서동,137
운서동 쓰레기 불법투기및 방치 조치요청,민원 내용:운서동 쓰레기 불법투기및 방치 조치요청아름다운 중구 위해 힘써주시는 여러...,답변에 감사드립니다.일단 동에서 할수있는 조치를 해주시기 바랍니다.(폐기물 불법투기...,운서동,266
관광버스 쓰레기무단 투기 단속요청,민원 내용:관광버스 쓰레기무단 투기 단속요청수고많으십니다. 중구 흰바위로15 신도시...,1. 우리 洞 발전에 관심을 가져주신 귀하께 감사의 말씀을 드립니다.2. 귀하께서 ...,운서동,397


In [250]:
document_embeddings = compute_doc_embeddings(Environmental)

In [252]:
# embedding example
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

영종도 운서역 종량제 10L 어디에 파나요 ? : [0.00199165940284729, -0.03413895517587662, 0.00398331880569458, -0.018205681815743446, -0.018456703051924706]... (1536 entries)


In [253]:
#tuple->list->dataframe으로 변경해야 csv형태로 저장 가능
example_entry1 = list(document_embeddings.items())
embedding_df = pd.DataFrame(example_entry1, columns=['title', 'embedding'])
print(embedding_df)

                       title  \
0  영종도 운서역 종량제 10L 어디에 파나요 ?   
1        앞 공터 쓰레기더미를 치워 주세요!   
2           운서동 두림타워 신축공사 민원   
3      운서동 쓰레기 불법투기및 방치 조치요청   
4         관광버스 쓰레기무단 투기 단속요청   
5           공동주거지 인근 노천소각 신고   
6       공항신도시 쓰레기 투기 관련 조치요청   
7                   쓰레기 수거요구   
8            쓰레기 처리 좀 해 주세요~   
9              복지정책을역행하는공무원!   

                                           embedding  
0  [0.00199165940284729, -0.03413895517587662, 0....  
1  [-0.008843394927680492, -0.01874910108745098, ...  
2  [-0.0018028655322268605, -0.020556490868330002...  
3  [-0.0006840603891760111, -0.01572529226541519,...  
4  [0.00010881438356591389, -0.011694789864122868...  
5  [-0.0017284720670431852, -0.010009659454226494...  
6  [-0.006901671178638935, -0.03658057004213333, ...  
7  [0.0022447637747973204, -0.014944544993340969,...  
8  [-0.0004525410768110305, 0.007362122181802988,...  
9  [-0.018691372126340866, -0.012628423050045967,...  


In [254]:
##임베딩값 저장
embedding_df.to_csv("embedding_운서동.csv", encoding='utf-8-sig',mode='w',index=False)

---

## 임베딩값 불러오기

In [8]:
import os
import pandas as pd

path = './' # CSV 파일이 있는 경로
files = os.listdir(path) # 해당 경로 내의 모든 파일 리스트
csv_files = [file for file in files if file.endswith('.csv') and file.startswith('embedding')] # CSV 파일만 추출

dfs = [] # 각각의 CSV 파일에서 읽은 데이터프레임을 저장할 리스트
for csv_file in csv_files:
    df = pd.read_csv(os.path.join(path, csv_file), index_col=None, header=0)
    dfs.append(df)

result_df = pd.concat(dfs, axis=0) # 모든 데이터프레임을 합침


In [9]:
result_df

,title,embedding
0,집 베란다 앞에서 피우는 담배때문에 힘들어요,"[0.0036535034887492657, -0.010342638939619064,..."
1,학교앞 담배꽁초,"[-0.0008675387944094837, -0.006503315176814794..."
2,신포시장 내 흡연 및 노상방뇨 건,"[-0.004503477830439806, -0.015920251607894897,..."
3,인천공항 금연구역 흡연문제,"[-0.003821880090981722, -0.007143765222281218,..."
4,방역 실시의 건,"[-0.013074032962322235, -0.020396584644913673,..."
...,...,...
13,지나다닐 때마다 너무 괴로워요.,"[-0.011721239425241947, -0.02832743525505066, ..."
14,금연구역 지정 및 cctv 설치요청,"[0.008484265767037868, -0.008444183506071568, ..."
15,"중구청 환경보호과, 주말 집에서 쉬고 싶습니다. 소음,먼지,진동 문제","[0.02148951031267643, 0.0043196361511945724, 0..."
16,신흥시장 점포 정화조 가능한지 문의합니다.,"[0.005075301509350538, -0.02015163190662861, 0..."


In [12]:
# 다시 df전의 원형으로 만들기
result_df['embedding'] = result_df['embedding'].apply(lambda x: [float(i) for i in x.strip('[]').split(',')])

r_document_embeddings = dict(zip(result_df['title'], result_df['embedding']))

In [135]:
#임베딩 저장한거 잘 작동되는 지 확인하고 싶다구링
query="불법주차 단속 안하나요? 횡단보도 앞뒤로 주차가 되어있어서 횡단보도를 건너는데 차가 오는지안오는지 보이지가 않아요. 어제 전동교회 앞에서 횡단보도를 건너다가 차가 갑자기 빠르게 지나가서 깜짝 놀랐고, 사고날뻔 했어요. 단속 좀 제대로 해주세요"
similar_complaints=order_document_sections_by_query_similarity(query, r_document_embeddings)[:5]

In [136]:
#유사민원 전체 내용 및 df 불러오기.
###유사민원 결과를 저장 값->set의 list형태
similar_complaints

[(0.9059462243363667, '소방도로에 불법 주차 차량 단속 및 주차 금지 표지 설치 요청'),
 (0.9045972263806786, '신포동 불법주정차'),
 (0.902606187015441, '인도위 불법주차 단속'),
 (0.901385507405047, '주택가에 세워진 주민의 차에 불법 주차딱지가 웬말인가?'),
 (0.8935640002553571, '항운아파트 화물차불법추자 정차')]

In [61]:
#유사도가 높은 순서대로 3개의 민원을 기반으로 prompt생성
def generate_prompts(similar_complaints, query):
    prompts = []
    for i in range(3):
        title = similar_complaints[i][1]
        prompt = f"-민원제목: {title}\n{df.loc[title][0]}\n-민원 답변: {df.loc[title][1]}\n-새로운 민원:{query}\n-민원 답변:"
        prompts.append(prompt)
        print(f"prompt{i+1}: {prompt}")
    return prompts

In [62]:
prompts = generate_prompts(similar_complaints, query)

prompt1: -민원제목: 불법주차 때문에 다니기가 힘듭니다
민원 내용:불법주차 때문에 다니기가 힘듭니다운남동 중구농협본점 뒷편은평일엔 항상 양쪽으로 주차가 되어 통행이 나도 어렵다양방향으로 진행시 한쪽이 비어있으면 양보하여 겨우 지나갈수 있지만그럴 공간이 없을 경우 정말 난감 불법주차된 차량사이로 사람이 갑자기 나타나면 정말 운전이 무서워집니다...비단 이 지역만의 문제가 기 때문에운남동 원룸이 많은 골목과 불법주차가 많은 지역에 불법주정차로 인한 피해가 없도록 지속적인 단속과 cctv설치 건의드립니다
-민원 답변: 지속관리를 하신다고 하셨는데...
현재까지도 개선이 안되고 있으며

농협앞 도로는 버스정류장이 있음에도
불법주정차가 만연한 상황입니다

그리고 불법주정차를 신고하려면
운전자가 이동한 상황이 되어버리고
또한 신고도중 시비가 생길 여지도 있을것 같습니다

cctv는 설치가 불가능한건가요??
-새로운 민원:불법주차 단속 안하나요? 횡단보도 앞뒤로 주차가 되어있어서 횡단보도를 건너는데 차가 오는지안오는지 보이지가 않아요. 어제 전동교회 앞에서 횡단보도를 건너다가 차가 갑자기 빠르게 지나가서 깜짝 놀랐고, 사고날뻔 했어요. 단속 좀 제대로 해주세요
-민원 답변:
prompt2: -민원제목: 중구청 뒤 불법 주차 단속 요청
민원 내용:중구청 뒤 불법 주차 단속 요청. 아침 저녁으로 중구청 뒷길을 차량으로 이용하고 있다. 신포동 일대의 주차 단속은 꽤 철저한 편입니다. 제가 아는 지인도 주차 구역이 아닌 곳에 주차 후 주차 위반으로 여러번 범칙금을 내는 것을 봤다. 주차 구역 이외의 지역에 주차 하는 것은 여러 사람에게 불편을 초래하는 일로 당연히 철저하게 단속되어야 . 하지만 중구청 뒷길은 왜 이중 주차 허용하는지 궁금. 매일 그 길을 이용하면서 차량들이 엉키고 큰 트럭이 힘겹게 후진을 하는 일이 다반사입니다. 가장 어이없는 일은 차량들이 엉켜서 극심한 정체 빚고 있었을 때 주차 단속을 나오신 분이 어떠한 차량도 주차 단속하지 않고 차 후진하라는 말만 하

In [82]:
print(prompts[0])

-민원제목: 불법주차 때문에 다니기가 힘듭니다
민원 내용:불법주차 때문에 다니기가 힘듭니다운남동 중구농협본점 뒷편은평일엔 항상 양쪽으로 주차가 되어 통행이 나도 어렵다양방향으로 진행시 한쪽이 비어있으면 양보하여 겨우 지나갈수 있지만그럴 공간이 없을 경우 정말 난감 불법주차된 차량사이로 사람이 갑자기 나타나면 정말 운전이 무서워집니다...비단 이 지역만의 문제가 기 때문에운남동 원룸이 많은 골목과 불법주차가 많은 지역에 불법주정차로 인한 피해가 없도록 지속적인 단속과 cctv설치 건의드립니다
-민원 답변: 지속관리를 하신다고 하셨는데...
현재까지도 개선이 안되고 있으며

농협앞 도로는 버스정류장이 있음에도
불법주정차가 만연한 상황입니다

그리고 불법주정차를 신고하려면
운전자가 이동한 상황이 되어버리고
또한 신고도중 시비가 생길 여지도 있을것 같습니다

cctv는 설치가 불가능한건가요??
-새로운 민원:불법주차 단속 안하나요? 횡단보도 앞뒤로 주차가 되어있어서 횡단보도를 건너는데 차가 오는지안오는지 보이지가 않아요. 어제 전동교회 앞에서 횡단보도를 건너다가 차가 갑자기 빠르게 지나가서 깜짝 놀랐고, 사고날뻔 했어요. 단속 좀 제대로 해주세요
-민원 답변:


In [162]:
def print_response (msg):
    completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=msg
                    )
    return completion['choices'][0]['message']['content']

In [163]:
#몇번째 생성된 prompt를 넣을 지
n=prompt1
messages = [
    {"role": "system", "content": "You should act as an assistant to the civil service officer.모든 대답은 한글로."},
    {"role": "user", "content" :"{} \n 위의 민원 답변 형식에 맞춰서 새로운 민원에 대한 답변을 생성해.".format(n)}
]
answer=print_response(messages)

In [164]:
print(n)
print("-----------------------------------------------------------")
print(answer)

-민원제목: 소방도로에 불법 주차 차량 단속 및 주차 금지 표지 설치 요청
민원 내용:소방도로에 불법 주차 차량 단속 및 주차 금지 표지 설치 요청동인천 내리교회 입구 소방도로에 불법 주차 차량으로 인해 사고 위험 등으로 생활에 큰 불편을 초래하는데도 전화 민원을 넣으면 그때만 단속하고나면  다른 차량들이 주차하는 악순환이 반복 되는데 근본적으로 주차 못하도록 안내 표지판 및 cctv 활용한 상시 단속으로 불법 주차하면 안된다는 사실을 인식시키는 근본적인 대책을 세워 주시기 바랍니다.대책에 대한 사항은 정식으로 과장급 이상의 책임자 결제가 포함된 공문서로 작성해서 통보해 주시기 바라며 작성된 사항의 이행여부에 따른 법적 잭임을 물을 수 있음을 알려드리며 명확한 처리 결과 알려 주시기 바랍니다. 
-민원 답변: 새올상담민원에 대하여 아래와 같이 답변하고자 합니다.1. 안녕하십니까? 우리 구정에 관심을 가지시고 의견을 주셔서 감사 드립니다.   귀하께서 제기하신 동인천 내리교회입구의 불법주정차에 대하여 답변드리겠습니다. 2. 귀하께서 요청하신 내리교회 앞 도로에 주정차금지 노면표시 및 주정차금지 표지판은 바로 설치 및 표시하도록 하겠습니다. 지속적인 불법주정차로 발생되는 문제와 관련하여 강력한 계도 및 단속을 실시하여 통행환경개선과 주정차질서 확립을 위해 최대한 노력하겠으며, 불법주정차 CCTV 설치와 관련하여 귀하께서 요청하신 사항을 적극적으로 수용·검토하여 상시적인 불법주정차가 단속될 수 있도록 최선을 다하겠습니다. 3. 불법주정차 CCTV 설치의 경우 예산이 수반되는 사항으로 즉시 설치가 어려운 점을 널리 이해하여 주시기 바라며, 기타 궁금하신 점이 있으시면 중구청 교통운수과(☏ 760-7684)로 연락주시면 성심성의껏 답변드리겠습니다. 감사합니다.붙임  새올상담민원 1부.  끝.\-새로운 민원:불법주차 단속 안하나요? 횡단보도 앞뒤로 주차가 되어있어서 횡단보도를 건너는데 차가 오는지안오는지 보이지가 않아요. 어제 전동교회 앞에서 횡단보도를 건너다가 차가 갑

---

## 답변생성 정리

In [42]:
#민원 데이터 불러오기
df= pd.read_csv('minwon.csv', sep=",", dtype=str)

#token수 확인
## gpt-3.5-turbo 모델에 맞는 token encoding model은 cl100k_base이다.
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

#contents 컬럼의 토큰 수를 계산하여 새로운 컬럼으로 만들어 붙임
df['tokens'] = df.content.apply(lambda x: num_tokens_from_string(x, 'cl100k_base'))
#del df['Unnamed: 0']

df = df.set_index(keys='title')

In [ ]:
df = df[df['tokens'] <= 1200]
df.head()

In [24]:
#임베딩 데이터 불러오기
path = './' 
files = os.listdir(path) 
csv_files = [file for file in files if file.endswith('.csv') and file.startswith('embedding')] # CSV 파일만 추출

dfs = []
for csv_file in csv_files:
    df = pd.read_csv(os.path.join(path, csv_file), index_col=None, header=0)
    dfs.append(df)

result_df = pd.concat(dfs, axis=0) 

# 다시 df전의 원형으로 만들기
result_df['embedding'] = result_df['embedding'].apply(lambda x: [float(i) for i in x.strip('[]').split(',')])

r_document_embeddings = dict(zip(result_df['title'], result_df['embedding']))

In [70]:
##유사민원 탐색 및 답변생성에 필요한 함수 정의

COMPLETIONS_MODEL = "gpt-3.5-turbo"
EMBEDDING_MODEL = "text-embedding-ada-002"

def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }

#코사인유사도
def vector_similarity(x: list[float], y: list[float]) -> float:
    
    return np.dot(np.array(x), np.array(y))

#새로운 입력값을 임베딩
def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:

    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

#유사도가 높은 순서대로 3개의 민원을 기반으로 prompt생성
def generate_prompts(similar_complaints, query):
    prompts = []
    for i in range(3):
        title = similar_complaints[i][1]
        prompt = f"-민원제목: {title}\n{df.loc[title][0]}\n-민원 답변: {df.loc[title][1]}\n-새로운 민원:{query}\n-민원 답변:"
        prompts.append(prompt)
        print("----------------------------------------------------")
        print(f"prompt{i+1}: {prompt}")
    return prompts

#답변받기
def print_response (msg):
    completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=msg
                    )
    return completion['choices'][0]['message']['content']

In [69]:
#임베딩 저장한거 잘 작동되는 지 확인
##새로운 민원
query="불법주차 단속 안하나요? 횡단보도 앞뒤로 주차가 되어있어서 횡단보도를 건너는데 차가 오는지안오는지 보이지가 않아요. 어제 전동교회 앞에서 횡단보도를 건너다가 차가 갑자기 빠르게 지나가서 깜짝 놀랐고, 사고날뻔 했어요. 단속 좀 제대로 해주세요"
similar_complaints=order_document_sections_by_query_similarity(query, r_document_embeddings)[:5]

#유사민원 전체 내용 및 df 불러오기.
###유사민원 결과를 저장 값->set의 list형태
print(similar_complaints)

#프롬프트 구조화 
prompts=[]
prompts = generate_prompts(similar_complaints, query)

[(0.9160962676747257, '불법주차 때문에 다니기가 힘듭니다'), (0.9128999333027236, '중구청 뒤 불법 주차 단속 요청'), (0.906932153124165, '불법 주정차 단속요청'), (0.9054444429322579, '소방도로에 불법 주차 차량 단속 및 주차 금지 표지 설치 요청'), (0.904498038184272, '신포동 불법주정차')]
prompt1: -민원제목: 불법주차 때문에 다니기가 힘듭니다
민원 내용:불법주차 때문에 다니기가 힘듭니다운남동 중구농협본점 뒷편은평일엔 항상 양쪽으로 주차가 되어 통행이 나도 어렵다양방향으로 진행시 한쪽이 비어있으면 양보하여 겨우 지나갈수 있지만그럴 공간이 없을 경우 정말 난감 불법주차된 차량사이로 사람이 갑자기 나타나면 정말 운전이 무서워집니다...비단 이 지역만의 문제가 기 때문에운남동 원룸이 많은 골목과 불법주차가 많은 지역에 불법주정차로 인한 피해가 없도록 지속적인 단속과 cctv설치 건의드립니다
-민원 답변: 지속관리를 하신다고 하셨는데...
현재까지도 개선이 안되고 있으며

농협앞 도로는 버스정류장이 있음에도
불법주정차가 만연한 상황입니다

그리고 불법주정차를 신고하려면
운전자가 이동한 상황이 되어버리고
또한 신고도중 시비가 생길 여지도 있을것 같습니다

cctv는 설치가 불가능한건가요??
-새로운 민원:불법주차 단속 안하나요? 횡단보도 앞뒤로 주차가 되어있어서 횡단보도를 건너는데 차가 오는지안오는지 보이지가 않아요. 어제 전동교회 앞에서 횡단보도를 건너다가 차가 갑자기 빠르게 지나가서 깜짝 놀랐고, 사고날뻔 했어요. 단속 좀 제대로 해주세요
-민원 답변:
prompt2: -민원제목: 중구청 뒤 불법 주차 단속 요청
민원 내용:중구청 뒤 불법 주차 단속 요청. 아침 저녁으로 중구청 뒷길을 차량으로 이용하고 있다. 신포동 일대의 주차 단속은 꽤 철저한 편입니다. 제가 아는 지인도 주차 구역이 아닌 곳에 주차 후 주차 위반으로 여러번 범칙금을 내는 것을 봤다. 주차 구

In [71]:
#몇번째 생성된 prompt를 넣을 지
n=prompts[0]
messages = [
    {"role": "system", "content": "You should act as an assistant to the civil service officer.모든 대답은 한글로."},
    {"role": "user", "content" :"{} \n 위의 민원 답변 형식에 맞춰서 새로운 민원에 대한 답변을 생성해.".format(n)}
]
answer=print_response(messages)

In [74]:
print(n)
print("------------------------------------------")
print(answer)

-민원제목: 불법주차 때문에 다니기가 힘듭니다
민원 내용:불법주차 때문에 다니기가 힘듭니다운남동 중구농협본점 뒷편은평일엔 항상 양쪽으로 주차가 되어 통행이 나도 어렵다양방향으로 진행시 한쪽이 비어있으면 양보하여 겨우 지나갈수 있지만그럴 공간이 없을 경우 정말 난감 불법주차된 차량사이로 사람이 갑자기 나타나면 정말 운전이 무서워집니다...비단 이 지역만의 문제가 기 때문에운남동 원룸이 많은 골목과 불법주차가 많은 지역에 불법주정차로 인한 피해가 없도록 지속적인 단속과 cctv설치 건의드립니다
-민원 답변: 지속관리를 하신다고 하셨는데...
현재까지도 개선이 안되고 있으며

농협앞 도로는 버스정류장이 있음에도
불법주정차가 만연한 상황입니다

그리고 불법주정차를 신고하려면
운전자가 이동한 상황이 되어버리고
또한 신고도중 시비가 생길 여지도 있을것 같습니다

cctv는 설치가 불가능한건가요??
-새로운 민원:불법주차 단속 안하나요? 횡단보도 앞뒤로 주차가 되어있어서 횡단보도를 건너는데 차가 오는지안오는지 보이지가 않아요. 어제 전동교회 앞에서 횡단보도를 건너다가 차가 갑자기 빠르게 지나가서 깜짝 놀랐고, 사고날뻔 했어요. 단속 좀 제대로 해주세요
-민원 답변:
------------------------------------------
안녕하세요. 시민 서비스 담당자입니다.

불법주차 단속에 대한 권고 및 주의사항을 운전자들에게 지속적으로 알리고 있지만, 여전히 일부 운전자들은 이를 무시하는 경우가 있습니다. 또한, 현재 해당 지역은 불법주차 신고가 많이 이루어지고 있어, 단속 인력을 배치하여 지속적인 단속을 하고 있습니다.

횡단보도 앞뒤로 주차된 차량에 대해서는, 안전한 보행 환경 조성을 위해 단속을 강화할 예정이며, 또한 해당 지역에 CCTV 설치를 검토하도록 하겠습니다.

시민 여러분의 건강하고 안전한 일상을 위해 항상 최선을 다하겠습니다. 감사합니다.


---

In [75]:
#몇번째 생성된 prompt를 넣을 지
n=prompts[1]
messages = [
    {"role": "system", "content": "You should act as an assistant to the civil service officer.모든 대답은 한글로."},
    {"role": "user", "content" :"{} \n 위의 민원 답변 형식에 맞춰서 새로운 민원에 대한 답변을 생성해.".format(n)}
]
answer=print_response(messages)

In [76]:
print(n)
print("------------------------------------------")
print(answer)

-민원제목: 중구청 뒤 불법 주차 단속 요청
민원 내용:중구청 뒤 불법 주차 단속 요청. 아침 저녁으로 중구청 뒷길을 차량으로 이용하고 있다. 신포동 일대의 주차 단속은 꽤 철저한 편입니다. 제가 아는 지인도 주차 구역이 아닌 곳에 주차 후 주차 위반으로 여러번 범칙금을 내는 것을 봤다. 주차 구역 이외의 지역에 주차 하는 것은 여러 사람에게 불편을 초래하는 일로 당연히 철저하게 단속되어야 . 하지만 중구청 뒷길은 왜 이중 주차 허용하는지 궁금. 매일 그 길을 이용하면서 차량들이 엉키고 큰 트럭이 힘겹게 후진을 하는 일이 다반사입니다. 가장 어이없는 일은 차량들이 엉켜서 극심한 정체 빚고 있었을 때 주차 단속을 나오신 분이 어떠한 차량도 주차 단속하지 않고 차 후진하라는 말만 하고선 가군요. 정말 화가  났다. 그 골목은 주차단속을 하지 않는 구역인가요이런 일이 하루이틀 저만의 문제가 라서 주변 사람들에게 이야기해 보니 그 중구청 뒷골목의 차량들 대부분이 중구청 직원 차량이라는 이야기 들었다그래서 단속하지 않는다는 이야기도 들었다. 유독 중구청 뒤만 단속하지 않는 것으로 보아 이런 이야기가 나올만 .. 물론 이것은 확인되지 않은 사실이지만  이것이 사실이라면 요즘같이 1인 1차량 시대에 직원들을 위한 주차장이 구비되어야 한다고 봅니다. 이러한 불편을 그 지역을 지나다니는 사람들이 떠안을 이유는 없다고 봅니다.  하나 이렇게 민원을 넣어 봤자 일주일 단속한 뒤  원상대로 돌아간다는 이야기도 들었다. 저가 아이핀 인증까지 힘겹게 받아가지고 여기에 구구절절 민원을 남기는 이유는 잠깐만 불편을 해소하고자 올리는 것이 아닙니다. 근본적으로 문제가 해결되어서 중구청 뒷길에 이중주차 문제 불법 주차 문제가 해결되길 원하는 바입니다. 매번 그 길을 이용하면서 불편함이 아주 극심. 중구 지역의 불법 주차 문제 단속하고 편리 도모하는 중구청에서 먼저 모범을 보여 주었으면 . 이 문제가 근본적으로 해결되지 않는다면 다른 곳에 도움을 받아 해결해 볼 방법을 도모하겠다.
-민원 답변